In [1]:
import numpy as np
import sklearn
import scipy.stats
import matplotlib
matplotlib.use('PDF')
%matplotlib inline
import matplotlib.pyplot as plt
import timeit
import pandas as pd
import math
import itertools
import time

import sklearn.neighbors
from sklearn.neighbors import (
    KernelDensity,
    KDTree,
)

In [2]:
d = 4
mu = np.zeros(d)
cov = np.identity(d)

In [4]:
rv = scipy.stats.multivariate_normal(mu, cov)
samples = rv.rvs(size=10000000)
s_pdfs = rv.pdf(samples)
q_p = np.percentile(s_pdfs, 10)
n = 100000

In [20]:
xs = np.arange(-2.5,2.5,0.5)
grid_points = []
for x1 in xs:
    for x2 in xs:
        for x3 in xs:
            for x4 in xs:
                grid_points.append(np.array([x1,x2,x3,x4]))
score_samples = np.array(grid_points)

In [21]:
score_pdfs = rv.pdf(score_points)

In [22]:
def eval(t, pdfs):
    predict = pdfs < t
    true = score_pdfs[:m] < t
    both = predict & true
    return (np.sum(both), np.sum(predict), np.sum(true))

# KDE

In [5]:
bw = (4/(d+4))**(1/(d+4)) * m ** (-1.0/(d+4))
bw

0.26591479484724945

In [24]:
kde = KernelDensity(
    bandwidth=bw,
    kernel='gaussian',
    algorithm='kd_tree',
    rtol=1e-3,
)
kde.fit(samples[:n])
kde_scores = kde.score_samples(score_samples)
kde_pdfs = np.exp(kde_scores)
kde_deltas = kde_pdfs-score_pdfs
print((np.mean(kde_deltas**2)))
print(eval(q_p, kde_pdfs))

9.19682472196e-08
(5123, 5172, 5283)


In [25]:
kde = KernelDensity(
    bandwidth=.1,
    kernel='gaussian',
    algorithm='kd_tree',
    rtol=1e-3,
)
kde.fit(samples[:n])
kde_scores = kde.score_samples(score_samples)
kde_pdfs = np.exp(kde_scores)
kde_deltas = kde_pdfs-score_pdfs
print((np.mean(kde_deltas**2)))
print(eval(q_p, kde_pdfs))

9.70259515685e-07
(4776, 5736, 5283)


In [26]:
kde = KernelDensity(
    bandwidth=.4,
    kernel='gaussian',
    algorithm='kd_tree',
    rtol=1e-3,
)
kde.fit(samples[:n])
kde_scores = kde.score_samples(score_samples)
kde_pdfs = np.exp(kde_scores)
kde_deltas = kde_pdfs-score_pdfs
print((np.mean(kde_deltas**2)))
print(eval(q_p, kde_pdfs))

3.0126117406e-07
(4935, 4935, 5283)


# Histogram

In [45]:
bw = 3.5 * 1 * (n) ** (-1.0/(2+d))
bw

0.5137297436677244

In [46]:
def calc_hist(bdelta):
    bins_1d = np.arange(-4, 4.1, bdelta)
    bins = np.array([bins_1d] * d)
    def to_bin(x):
        idxs = np.searchsorted(bins_1d, x)
        return np.minimum(np.maximum(idxs,0),len(bins_1d)-2)
    H, edges = np.histogramdd(samples[:n], bins=bins, normed=True)
    hist_scores = [
        H[tuple(to_bin(cursample))] for cursample in score_samples[:m]
    ]
    return hist_scores

In [47]:
hist_scores = calc_hist(0.5)
deltas = hist_scores-score_pdfs[:m]
print((np.mean(deltas**2)))
print(eval(q_p, hist_scores))

1.45248185195e-06
(4587, 5339, 5283)


In [48]:
hist_scores = calc_hist(0.3)
deltas = hist_scores-score_pdfs[:m]
print((np.mean(deltas**2)))
print(eval(q_p, hist_scores))

4.40642547093e-06
(4330, 5896, 5283)


In [49]:
hist_scores = calc_hist(0.7)
deltas = hist_scores-score_pdfs[:m]
print((np.mean(deltas**2)))
print(eval(q_p, hist_scores))

7.90726644216e-06
(3690, 5508, 5283)


# KNN

In [27]:
bk = n ** (4/(4+d))
bk

316.22776601683796

In [28]:
import scipy.special
def vol_sphere(n,r):
    return (math.pi)**(n/2)/(scipy.special.gamma(n/2+1)) * r**n
vsphere = vol_sphere(4,1)
tree = KDTree(samples[:n])

In [29]:
def calc_knn(k):
    distances, _ = tree.query(score_samples[:m], k=k)
    knndistances = distances[:,k-1]
    knn_scores = (k/n) / (vsphere * np.power(knndistances, d))
    return knn_scores

In [52]:
knn_scores = calc_knn(316)
knn_deltas = knn_scores-score_pdfs[:m]
print((np.mean(knn_deltas**2)))
print(eval(q_p, knn_scores))

6.46581544256e-08
(4812, 4812, 5283)


In [43]:
knn_scores = calc_knn(50)
knn_deltas = knn_scores-score_pdfs[:m]
print((np.mean(knn_deltas**2)))
print(eval(q_p, knn_scores))

2.45208613925e-07
(5120, 5155, 5283)


In [44]:
knn_scores = calc_knn(1000)
knn_deltas = knn_scores-score_pdfs[:m]
print((np.mean(knn_deltas**2)))
print(eval(q_p, knn_scores))

1.05043666337e-07
(4300, 4300, 5283)
